In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19635755
paper_name = 'jo_vulpe_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_excel('raw_data/Table_S2.xlsx', sheet_name='Table 1', skiprows=4, header=None)
original_data2 = pd.read_excel('raw_data/Table_S3.xlsx', sheet_name='Table 1', skiprows=4, header=None)

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 953 x 10
Original data dimensions: 669 x 10


In [7]:
original_data1.head()

,0,1,2,3,4,5,6,7,8,9
0,YJL101C,GSH1,-3.45,-2.1,-4.45,-4.3,NaN,-4.9,5,NaN
1,YFR036W,CDC26,-1.7,NaN,-2.8,-4.15,-5.7,-3.9,5,NaN
2,YMR275C,BUL1,-1.4,-2.25,-3.15,-4.55,NaN,-4.5,5,NaN
3,YDR135C,YCF1,-0.8,NaN,-1.4,-2.85,-4.8,-4.2,5,NaN
4,YPL066W,NaN,NaN,-1.35,-1.7,-2.45,-4.75,-3.3,5,NaN


In [8]:
original_data1.columns = ['orfs','genes','37uM_5gen','75uM_5gen','150uM_5gen','37uM_15gen','75uM_15gen','150uM_15gen', 'num_hits','t']
original_data2.columns = ['orfs','genes','75uM_5gen','150uM_5gen','300uM_5gen','75ug_15gen','150uM_15gen','300uM_15gen', 'num_hits','t']

In [9]:
original_data1['orfs'] = original_data1['orfs'].astype(str)
original_data2['orfs'] = original_data2['orfs'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data1['orfs'] = clean_orf(original_data1['orfs'])
original_data2['orfs'] = clean_orf(original_data2['orfs'])

In [11]:
# Translate to ORFs 
original_data1['orfs'] = translate_sc(original_data1['orfs'], to='orf')
original_data2['orfs'] = translate_sc(original_data2['orfs'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orfs'])
print(original_data1.loc[~t,])

            orfs genes      37uM_5gen 75uM_5gen 150uM_5gen      37uM_15gen  \
index_input                                                                  
43           ORF  Gene  5 generations       NaN        NaN  15 generations   
44           NAN   NaN         37.5µM      75µM      150µM          37.5µM   
102          ORF  Gene  5 generations       NaN        NaN  15 generations   
103          NAN   NaN         37.5µM      75µM      150µM          37.5µM   
161          ORF  Gene  5 generations       NaN        NaN  15 generations   
162          NAN   NaN         37.5µM      75µM      150µM          37.5µM   
220          ORF  Gene  5 generations       NaN        NaN  15 generations   
221          NAN   NaN         37.5µM      75µM      150µM          37.5µM   
279          ORF  Gene  5 generations       NaN        NaN  15 generations   
280          NAN   NaN         37.5µM      75µM      150µM          37.5µM   
338          ORF  Gene  5 generations       NaN        NaN  15 g

In [13]:
original_data1 = original_data1.loc[t,]

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orfs'])
print(original_data2.loc[~t,])

            orfs genes      75uM_5gen 150uM_5gen 300uM_5gen      75ug_15gen  \
index_input                                                                   
44           ORF  Gene  5 generations        NaN        NaN  15 generations   
45           NAN   NaN         37.5µM       75µM      150µM          37.5µM   
103          ORF  Gene  5 generations        NaN        NaN  15 generations   
104          NAN   NaN         37.5µM       75µM      150µM          37.5µM   
162          ORF  Gene  5 generations        NaN        NaN  15 generations   
163          NAN   NaN         37.5µM       75µM      150µM          37.5µM   
221          ORF  Gene  5 generations        NaN        NaN  15 generations   
222          NAN   NaN         37.5µM       75µM      150µM          37.5µM   
280          ORF  Gene  5 generations        NaN        NaN  15 generations   
281          NAN   NaN         37.5µM       75µM      150µM          37.5µM   
339          ORF  Gene  5 generations        NaN    

In [15]:
original_data2 = original_data2.loc[t,]

In [16]:
original_data1.set_index('orfs', inplace=True)
original_data2.set_index('orfs', inplace=True)
original_data1.index.name='orf'
original_data2.index.name='orf'

In [17]:
original_data1 = original_data1[['37uM_5gen','75uM_5gen','150uM_5gen','37uM_15gen','75uM_15gen','150uM_15gen']].copy()
original_data2 = original_data2[['75uM_5gen','150uM_5gen','300uM_5gen','75ug_15gen','150uM_15gen','300uM_15gen']].copy()

In [18]:
original_data1 = original_data1.apply(pd.to_numeric, axis=1, errors='coerce')
original_data2 = original_data2.apply(pd.to_numeric, axis=1, errors='coerce')

In [19]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [20]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_mma', rsuffix='_as')

In [22]:
original_data[original_data.isnull()] = 0

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [16656,16655,16654,16653,16652,16651,16659,16658,16657,16662,16661,16660]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,16656,16655,16654,16653,16652,16651,16659,16658,16657,16662,16661,16660
data_type,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,
YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.5,0.0
YAL011W,0.0,0.0,-1.3,0.0,-3.2,-2.6,0.0,0.0,-2.2,-1.5,-1.6,-5.0
YAL013W,0.0,0.0,0.0,0.0,1.7,3.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL019W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.4,0.0,0.0,-1.5
YAL020C,0.0,0.8,0.0,0.0,1.1,1.3,0.0,0.0,0.0,0.0,0.0,1.5


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16656,16655,16654,16653,16652,16651,16659,16658,16657,16662,16661,16660
,data_type,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.5,0.0
9,YAL011W,0.0,0.0,-1.3,0.0,-3.2,-2.6,0.0,0.0,-2.2,-1.5,-1.6,-5.0
11,YAL013W,0.0,0.0,0.0,0.0,1.7,3.0,0.0,0.0,0.0,0.0,0.0,0.0
17,YAL019W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.4,0.0,0.0,-1.5
18,YAL020C,0.0,0.8,0.0,0.0,1.1,1.3,0.0,0.0,0.0,0.0,0.0,1.5


In [29]:
data.shape

(1166, 12)

# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16656,16655,16654,16653,16652,16651,16659,16658,16657,16662,...,16654,16653,16652,16651,16659,16658,16657,16662,16661,16660
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012658,-0.062132,-0.069763,-0.011271,0.16155,0.091199,-0.090022,-0.07344,-3.214907,-0.040131
9,YAL011W,0.0,0.0,-1.3,0.0,-3.2,-2.6,0.0,0.0,-2.2,-1.5,...,-4.642052,-0.062132,-5.460234,-2.648660,0.16155,0.091199,-4.703398,-5.81335,-3.422930,-7.680487
11,YAL013W,0.0,0.0,0.0,0.0,1.7,3.0,0.0,0.0,0.0,0.0,...,0.012658,-0.062132,2.793925,3.031871,0.16155,0.091199,-0.090022,-0.07344,-0.094556,-0.040131
17,YAL019W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.4,0.0,...,0.012658,-0.062132,-0.069763,-0.011271,0.16155,0.091199,-3.025807,-0.07344,-0.094556,-2.332238
18,YAL020C,0.0,0.8,0.0,0.0,1.1,1.3,0.0,0.0,0.0,0.0,...,0.012658,-0.062132,1.783211,1.307424,0.16155,0.091199,-0.090022,-0.07344,-0.094556,2.251976


# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/12 [00:00<?, ?it/s]

Deleting all datasets for PMID 19635755...
Inserting the new data...


100%|██████████| 12/12 [00:24<00:00,  2.08s/it]

Updating the data_modified_on field...
